In [1]:
import netket,time,numpy
from honeycomb import *

In [2]:
#This cell will show how to calc gs energy by direct diagnolize
size=(2,2)
#get Hamiltonian, you can use hamiltonian.hilbert to get its Hilbert space
hamiltonian=get_hamiltonian(size)
#exact_diag will print and also return its eigenvalues(and eigenvectors)
exact_diag(hamiltonian,first_n=1,compute_eigenvectors=False)

20/03/18 15:35:17 540<get_graph:113,DEBUG> 
x_edge number: 4, [[1, 2, 0], [3, 0, 0], [5, 6, 0], [7, 4, 0]]
y_edge number: 4, [[0, 1, 1], [2, 3, 1], [4, 5, 1], [6, 7, 1]]
z_edge number: 4, [[1, 4, 2], [3, 6, 2], [5, 0, 2], [7, 2, 2]]
graph spinor number: 8, edge number: 12, is_bipartite: True
hilbert size: 8, n_states: 256
exact_diag used 0 s with result:
 -6.9282032303,


In [3]:
#This cell will show how to calc gs energy useing rbm
alpha=2
lr=0.03
n_iter=600 #should be at least 600
output_prefix="calc_gs_example"
ma=netket.machine.RbmSpin(hilbert=hamiltonian.hilbert,alpha=alpha)
ma.init_random_parameters(sigma=0.1)
#t is the time consumed by rbm
#the results will write in the folder of script
t=rbm(hamiltonian,ma,output_prefix,learning_rate=lr,n_iter=n_iter,n_samples=400)

20/03/18 15:35:20 033<rbm:196,INFO> machine has 152 parameters
20/03/18 15:35:20 038<rbm:197,INFO> rbm start with 0kx0k lr=0.030 df=1.0000
20/03/18 15:36:42 359<rbm:205,INFO> optimize machine takes 82.319573s


In [4]:
#This cell will show how to flip a rbm and measure
ma.init_random_parameters(sigma=0.1) #re-random rbm so that sz is not too small so that you can see the change
sz0=netket.operator.LocalOperator(hamiltonian.hilbert,numpy.asarray([[1.0,0.0],[0.0,-1.0]]),[0])
#measure will print some results and also return more infos
m1=measure(hamiltonian,ma,{"sz0":sz0},n_iter=100,n_samples=1000)
#2*2*2*2 is the number of hidden nodes of this machine
ma.parameters=flip_spins(ma,2*2*2*2,((0,"xz"),(1,"yz")))
m2=measure(hamiltonian,ma,{"sz0":sz0},n_iter=100,n_samples=1000)

20/03/18 15:37:15 698<measure:222,INFO> begin measure 1 obervables with 1000x100
20/03/18 15:37:39 502<measure:226,INFO> measuring 1 observables takes 23.802260s
20/03/18 15:37:39 511<measure:249,INFO> Energy: -2.865224(0.009507)
20/03/18 15:37:39 512<measure:249,INFO> sz0: 0.112379(0.003625)
20/03/18 15:37:39 514<flip_spins:280,INFO> n_v: 8, n_h: 16, n_par: 152
20/03/18 15:37:39 516<flip_spins:302,INFO> fliped ((0, 'xz'), (1, 'yz'))
20/03/18 15:37:39 517<measure:222,INFO> begin measure 1 obervables with 1000x100
20/03/18 15:38:03 349<measure:226,INFO> measuring 1 observables takes 23.829209s
20/03/18 15:38:03 356<measure:249,INFO> Energy: -1.397078(0.009138)
20/03/18 15:38:03 357<measure:249,INFO> sz0: -0.111230(0.003724)
